In [1]:
import pandas as pd
import numpy as np
import os, glob
from tqdm import tqdm

In [2]:
food=pd.read_csv('./food.csv',encoding='utf-8')
del food['Unnamed: 0']
food.head(3)

,리뷰내용,리뷰점수
0,맛있어서 재구매 했어요🤤 집에 닭가슴살이 차고 넘치긴 한데 하도 먹으니 질려서 요즘...,5
1,요새 물가가 너무 비싸서 할인 찾아 삼만리..이벤트 찾아 삼만리.. 하지만 돌고 돌...,5
2,이곳저곳 비교하다가 리뷰가 좋아서 구매하게 되었어요~저염스팀 닭가슴살은 조각으로 나...,5


In [3]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95579 entries, 0 to 95578
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   리뷰내용    95579 non-null  object
 1   리뷰점수    95579 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [4]:
data=food['리뷰내용']
target=food['리뷰점수']

In [5]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.2,random_state=42,stratify=target)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 텍스트 데이터를 TF-IDF로 변환
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(train_x)
X_test_tfidf = vectorizer.transform(test_x)

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [19]:
# 모델 생성
rf=RandomForestRegressor(max_depth=100,n_estimators=10)
rf.fit(X_train_tfidf,train_y)


RandomForestRegressor(max_depth=100, n_estimators=10)

In [20]:
# 테스트 세트로 예측
y_pred = rf.predict(X_test_tfidf)

In [21]:
# 모델 평가
mse = mean_squared_error(test_y, y_pred)
print(f'cv => Mean Squared Error: {mse}')

cv => Mean Squared Error: 0.32846604638265675


In [24]:
new_review = ["좋네요. 앞으로 종종 이용할게요"]  
new_review_tfidf = vectorizer.transform(new_review)

# 모델을 사용하여 예측
predicted_rating = rf.predict(new_review_tfidf)
rf_rating=rf.predict(new_review_tfidf)

print(f"예측된 리뷰평점: {rf_rating}")

예측된 리뷰평점: [4.8287525]


In [27]:
new_review = ["배달이 너무 늦고, 신선도가 떨어져요.. 최악이에요"]  
new_review_tfidf = vectorizer.transform(new_review)

# 모델을 사용하여 예측
predicted_rating = rf.predict(new_review_tfidf)
rf_rating=rf.predict(new_review_tfidf)

print(f"예측된 리뷰평점: {rf_rating}")

예측된 리뷰평점: [4.85040478]


### 낮은 평점 예측 잘 안됨  
Why? 높은 별점의 양이 상대적으로 훨씬 많은 데이터 불균형으로 인해